### Imports

In [264]:
import torch
from torch import nn
from torch import Tensor
torch.cuda.is_available()

True

In [265]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd

## Dataset

In [266]:
# For linux users:
# !wget -O birds.tgz https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1
# !tar zxvf birds.tgz

In [267]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
# from tqdm import tqdm

class CustomDataset(Dataset):
    def __init__(self, annotations, bboxes, dataset_path, img_size):
        self.annotations = annotations
        self.bboxes = bboxes
        self.dataset_path = dataset_path
        self.img_size = img_size

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_id = self.annotations.iloc[idx]['id']
        image_path = self.annotations.iloc[idx]['path']
        bbox = self.bboxes[self.bboxes['id'] == image_id].iloc[0]

        img = cv2.imread(self.dataset_path + "images/" + image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img[bbox['y']:bbox['y'] + bbox['height'], bbox['x']:bbox['x'] + bbox['width'], :]
        img = cv2.resize(img, (self.img_size, self.img_size))
        img = torch.tensor(img, dtype=torch.float32) / 127.5 - 1

        return img

dataset_path = 'CUB_200_2011/'
IMG_SIZE = 224  # Change according to your requirements
BATCH_SIZE = 32  # Change according to your requirements

# Read txt files containing info about the bounding boxes and the file paths
bboxes = pd.read_csv(dataset_path + 'bounding_boxes.txt', sep=" ", names=["id", "x", "y", "width", "height"]).astype(int)
annotations = pd.read_csv(dataset_path + 'images.txt', sep=" ", names=["id", "path"])

# Create the dataset
custom_dataset = CustomDataset(annotations, bboxes, dataset_path, IMG_SIZE)

# Define the number of samples for training and testing
limit_training = 10000
test = 300

# Split the dataset into training and testing
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [limit_training, test])

# Create DataLoader objects for training and testing
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(train_loader, test_loader)#store in array

## GAN Models

In [268]:
IMG_SIZE=64
BATCH_SIZE=64
LATENT_DIM=100

### Downsampling / Upsampling

In [269]:

class DownsampleBlock(nn.Module):
    def __init__(self, in_channels: int, filters: int, size: int, strides: int = 2, padding: str ='same', use_bn: bool = True) -> None:
        super().__init__()
        self.convolution = nn.Conv2d(in_channels, filters, kernel_size=size, stride=strides, padding=1)
        self.use_bn = use_bn
        self.batchNorm = nn.BatchNorm2d(filters)
        self.activation = nn.LeakyReLU(negative_slope=0.2)
        
    def forward(self, x: torch.Tensor) -> Tensor:
        x = self.convolution(x)
        if self.use_bn:
            x = self.batchNorm(x)
        x = self.activation(x)
        return x
    
class UpsampleBlock(nn.Module):
    def __init__(self, in_channels: int, filters: int, size: int, strides: int = 2, padding: str ='same', use_bn: bool = True) -> None:
        super().__init__()
        self.convolution = nn.ConvTranspose2d(in_channels, filters, kernel_size=size, stride=strides, padding=1)
        self.use_bn = use_bn
        self.batchNorm = nn.BatchNorm2d(filters)
        self.activation = nn.LeakyReLU(negative_slope=0.2)
        
    def forward(self, x: torch.Tensor) -> Tensor:
        x = self.convolution(x)
        if self.use_bn:
            x = self.batchNorm(x)
        x = self.activation(x)
        return x
        

### GAN Generator

In [270]:
class Generator(nn.Module):
    def __init__(self, input_size = LATENT_DIM) -> None:
        super().__init__()
        LINEAR_OUTPUT_SIZE = 4*4*256
        
        self.linear = nn.Linear(input_size, LINEAR_OUTPUT_SIZE)
        self.batchNorm = nn.BatchNorm1d(LINEAR_OUTPUT_SIZE)
        self.activation = nn.ReLU()
        self.upsampleBlock1 = UpsampleBlock(in_channels=256, filters=512, size=4)
        self.upsampleBlock2 = UpsampleBlock(in_channels=512, filters=256, size=4)
        self.upsampleBlock3 = UpsampleBlock(in_channels=256, filters=128, size=4)
        self.convTranspose2d = nn.ConvTranspose2d(in_channels=128, out_channels=3, kernel_size=4, stride=2, padding=2)
        
    def forward(self, x: torch.Tensor) -> Tensor: 
        # x - wektor szumu
        x = self.linear(x)
        x = self.batchNorm(x)
        x = self.activation(x)
        x = x.view(-1, 256, 4, 4) #reshape -> channel 4x4 image
        x = self.upsampleBlock1(x)
        x = self.upsampleBlock2(x)
        x = self.upsampleBlock3(x)
        x = self.convTranspose2d(x)
        return x

### GAN Discriminator

In [271]:
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.downsampleBlock1 = DownsampleBlock(in_channels=3, filters=64, size=4, strides=2)
        self.downsampleBlock2 = DownsampleBlock(in_channels=64, filters=128, size=4, strides=2)
        self.downsampleBlock3 = DownsampleBlock(in_channels=128, filters=256, size=4, strides=2)
        self.dropout = nn.Dropout(0.4)
        self.dense = nn.Linear(256*8*8,1)
        self.activation = nn.Sigmoid()
        
    def forward(self, x: torch.Tensor) -> Tensor:
        x = self.downsampleBlock1(x)
        x = self.downsampleBlock2(x)
        x = self.downsampleBlock3(x)
        print(x.shape)
        x = x.view(-1, 256*8*8)
        x = self.dropout(x)
        x = self.dense(x)
        x = self.activation(x)
        return x

## NN Instance

In [272]:
generator = Generator()
discriminator = Discriminator()

generator.forward(torch.randn(2, LATENT_DIM)).shape
discriminator.forward(torch.randn(2, 3, 64, 64)).shape

torch.Size([2, 256, 8, 8])


torch.Size([2, 1])